# Robotics Toolbox Example
Adapted based on example from https://github.com/jhavl/swift/blob/master/README.md

## Set up a simulation with a Franka Emika "Panda" robot

In [1]:
# Imports
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
from swift import Swift
from dataclasses import dataclass

In [2]:
# Settings (change them as you like)

@dataclass
class Params:
    # How many steps to simulate for each second. For visualization, 20 is a
    # good trade-off.
    simulation_frequency_in_hz: float = 20
    # If you have sufficient screen space to arrange two tabs side-by-side,
    # opening the simulation in a new tab ("True") makes it easier to see
    # what the robot is doing after having scrolled down in the notebook.
    open_simulation_in_new_tab: bool = False 

# instantiate with default values
params = Params()

In [ ]:
# Create simulation with Panda robot
env = Swift()
env.launch(realtime=True, browser="notebook" if not params.open_simulation_in_new_tab else None)
#env.launch(realtime=True, browser="notebook" if not params.open_simulation_in_new_tab else None, comms='rtc')  # Alternative Communictions
robot = rtb.models.Panda()
robot.q = robot.qr  # set robot joints to some default pose (here: called "qr" = "joint space, ready")
env.add(robot)

Exception in thread Thread-5 (SwiftSocket):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/daab/code/teaching/robotics-i/venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/daab/code/teaching/robotics-i/venv/lib/python3.10/site-packages/swift/SwiftRoute.py", line 296, in __init__
    start_server = websockets.serve(self.serve, "localhost", port)
  File "/home/daab/code/teaching/robotics-i/venv/lib/python3.10/site-packages/websockets/asyncio/server.py", line 737, in __init__
    self.server = Server(
  File "/home/daab/code/teaching/robotics-i/venv/lib/python3.10/site-packages/websockets/asyncio/server.py", line 281, in __init__
    self.loop = asyncio.get_running_loop()
RuntimeError: no running event loop


You should now see a Panda robot arm in a 3D simulation (it might take a while to load).

## Robot State and Forward Kinematics

To get familiar how to interact with the simulated robot, we query its current joint configuration.

In [4]:
joint_configuration = robot.q
print(joint_configuration)

[ 0.         -0.3         0.         -2.2         0.          2.
  0.78539816]


As you can see, the robot has seven joints. Now, we calculate the forward kinematics to get to know where the end effector is located in task space.

In [5]:
end_effector_pose_in_task_space = robot.fkine(joint_configuration)
print(end_effector_pose_in_task_space)

   0.995     0         0.09983   0.484     
   0        -1         0         0         
   0.09983   0        -0.995     0.4126    
   0         0         0         1         



The endeffector is located around 41 cm above the xy-plane. It is about 48cm away form the origin in x-direction, and 0 cm in y-direction.

## Moving the Robot

We will see in a later exercise how we can control the robot, to make it reach a target pose. For the moment, we just use the following function as-is:

In [6]:
# Define easy-to-use function to go to a target pose, defined in task space
def move_end_effector_to(target_pose):
    arrived = False
    while not arrived:
        task_space_vel, arrived = rtb.p_servo(robot.fkine(robot.q), target_pose, 1)
        joint_space_vel = np.linalg.pinv(robot.jacobe(robot.q)) @ task_space_vel
        robot.qd = joint_space_vel
        env.step(1 / params.simulation_frequency_in_hz)

Next, we define some target poses for the end effector in task space, so that we can easily switch between them. Pose_1 is a default pose, while pose_2 is translated by (0.2, 0.2, 0.45) relative to the default pose.

In [7]:
pose_1 = robot.fkine(robot.qr)
pose_2 = robot.fkine(robot.qr) * sm.SE3.Tx(0.2) * sm.SE3.Ty(0.2) * sm.SE3.Tz(0.45)
print(f'Pose 1:\n{pose_1}')
print(f'Pose 2:\n{pose_2}')

Pose 1:
   0.995     0         0.09983   0.484     
   0        -1         0         0         
   0.09983   0        -0.995     0.4126    
   0         0         0         1         

Pose 2:
   0.995     0         0.09983   0.728     
   0        -1         0        -0.2       
   0.09983   0        -0.995    -0.01516   
   0         0         0         1         



Now, we can request the robot to move between these poses:

In [8]:
move_end_effector_to(pose_2)

In [9]:
move_end_effector_to(pose_1)

In [10]:
move_end_effector_to(pose_2)